In [1]:
%pip install ../..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=20441 sha256=5d709bad00bf113b10e40c5b7d3d96dd02c9b3b1693efe3538a5e7e3f57e0d37
  Stored in directory: /tmp/pip-ephem-wheel-cache-vv3dnr4c/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1


In [1]:
import torch 

device = torch.device("cuda:3")

In [2]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_SMALL

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:00<00:00, 11533.26it/s]


In [3]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.datasets import GroupLabeledDatasetWrapper

robust_trainset = GroupLabeledDatasetWrapper(trainset, trainset.group_partition)

In [10]:
from torch.optim import SGD
from spuco.models import model_factory
from spuco.robust_train import GroupDRO

model = model_factory("mlp", trainset[0][0].shape, len(classes)).to(device)
group_dro = GroupDRO(
    model=model,
    num_epochs=5,
    trainset=robust_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [11]:
group_dro.train()

Epoch 2:  14%|█▍        | 107/751 [00:01<00:09, 65.61batch/s, accuracy=95.3125%, loss=0.281]

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 99.05437352245863
Group (0, 1) Test Accuracy: 91.725768321513
Group (0, 2) Test Accuracy: 93.14420803782505
Group (0, 3) Test Accuracy: 88.4160756501182
Group (0, 4) Test Accuracy: 93.14420803782505
Group (1, 0) Test Accuracy: 94.62102689486552
Group (1, 1) Test Accuracy: 97.31051344743277
Group (1, 2) Test Accuracy: 89.70588235294117
Group (1, 3) Test Accuracy: 65.19607843137256
Group (1, 4) Test Accuracy: 72.05882352941177
Group (2, 0) Test Accuracy: 64.0
Group (2, 1) Test Accuracy: 88.53333333333333
Group (2, 2) Test Accuracy: 96.53333333333333
Group (2, 3) Test Accuracy: 88.53333333333333
Group (2, 4) Test Accuracy: 54.54545454545455
Group (3, 0) Test Accuracy: 67.08542713567839
Group (3, 1) Test Accuracy: 62.97229219143577
Group (3, 2) Test Accuracy: 86.39798488664988
Group (3, 3) Test Accuracy: 96.97732997481108
Group (3, 4) Test Accuracy: 89.67254408060454
Group (4, 0) Test Accuracy: 88.41309823677582
Group (4, 1) Test Accuracy: 66.49874055415617
Grou

{(0, 0): 99.05437352245863,
 (0, 1): 91.725768321513,
 (0, 2): 93.14420803782505,
 (0, 3): 88.4160756501182,
 (0, 4): 93.14420803782505,
 (1, 0): 94.62102689486552,
 (1, 1): 97.31051344743277,
 (1, 2): 89.70588235294117,
 (1, 3): 65.19607843137256,
 (1, 4): 72.05882352941177,
 (2, 0): 64.0,
 (2, 1): 88.53333333333333,
 (2, 2): 96.53333333333333,
 (2, 3): 88.53333333333333,
 (2, 4): 54.54545454545455,
 (3, 0): 67.08542713567839,
 (3, 1): 62.97229219143577,
 (3, 2): 86.39798488664988,
 (3, 3): 96.97732997481108,
 (3, 4): 89.67254408060454,
 (4, 0): 88.41309823677582,
 (4, 1): 66.49874055415617,
 (4, 2): 63.727959697733,
 (4, 3): 89.39393939393939,
 (4, 4): 98.23232323232324}

In [ ]:
evaluator.worst_group_accuracy

((2, 4), 54.54545454545455)

In [ ]:
evaluator.average_accuracy

97.55885932606213

In [ ]:
evaluator.evaluate_spurious_attribute_prediction()

28.44